In [2]:
import functools
import matplotlib.pyplot as plt
import numpy as np
import torch
import seaborn as sns

from network import Net, NetDiscovery
from diff_equations import *

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

sns.set_theme()
torch.manual_seed(42)

In [3]:
np.random.seed(10)

Tenv = 25
T0 = 100
R = 0.005
times = np.linspace(0, 1000, 1000)
eq = functools.partial(cooling_law, Tenv=Tenv, T0=T0, R=R)
temps = eq(times)

# Make training data
t = np.linspace(0, 300, 10)
T = eq(t) + 2 * np.random.randn(10)

# plt.plot(times, temps)
# plt.plot(t, T, 'o')
# plt.legend(['Equation', 'Training data'])
# plt.ylabel('Temperature (C)')
# plt.xlabel('Time (s)')

In [ ]:
net = Net(1, 1, loss2=None, epochs=20000, lr=1e-5).to(DEVICE)

losses = net.fit(t, T)

# plt.plot(losses)
# plt.yscale('log')

Epoch 0/20000, loss: 4713.88
Epoch 2000/20000, loss: 2510.38
Epoch 4000/20000, loss: 2143.25
Epoch 6000/20000, loss: 727.66
Epoch 8000/20000, loss: 88.30
Epoch 10000/20000, loss: 1.40
Epoch 12000/20000, loss: 0.35
Epoch 14000/20000, loss: 0.34
Epoch 16000/20000, loss: 0.33
Epoch 18000/20000, loss: 0.33


In [ ]:
def l2_reg(model: torch.nn.Module):
    return torch.sum(sum([p.pow(2.0) for p in model.parameters()]))

In [ ]:
netreg = Net(1, 1, loss2=l2_reg, epochs=20000, lr=1e-4, loss2_weight=1).to(DEVICE)

losses = netreg.fit(t, T)

Epoch 0/20000, loss: 11219.99
Epoch 2000/20000, loss: 3915.88
Epoch 4000/20000, loss: 2451.49
Epoch 6000/20000, loss: 1647.56
Epoch 8000/20000, loss: 1229.45
Epoch 10000/20000, loss: 1030.59
Epoch 12000/20000, loss: 938.03
Epoch 14000/20000, loss: 858.53
Epoch 16000/20000, loss: 786.94
Epoch 18000/20000, loss: 722.30


In [7]:
predsreg = netreg.predict(times)

preds = net.predict(times)

In [ ]:
def physics_loss(model: torch.nn.Module):
    ts = (
        torch.linspace(
            0,
            1000,
            steps=1000,
        )
        .view(-1, 1)
        .requires_grad_(True)
        .to(DEVICE)
    )
    temps = model(ts)
    dT = grad(temps, ts)[0]
    pde = R * (Tenv - temps) - dT

    return torch.mean(pde**2)

In [ ]:
net = Net(1, 1, loss2=physics_loss, epochs=30000, loss2_weight=1, lr=1e-5).to(DEVICE)

losses = net.fit(t, T)

Epoch 0/30000, loss: 4775.84
Epoch 3000/30000, loss: 2438.49
Epoch 6000/30000, loss: 505.04
Epoch 9000/30000, loss: 1.70
Epoch 12000/30000, loss: 0.55
Epoch 15000/30000, loss: 0.47
Epoch 18000/30000, loss: 0.38
Epoch 21000/30000, loss: 0.34
Epoch 24000/30000, loss: 0.33
Epoch 27000/30000, loss: 0.34


In [11]:
preds = net.predict(times)

In [ ]:
def physics_loss_discovery(model: torch.nn.Module):
    ts = (
        torch.linspace(
            0,
            1000,
            steps=1000,
        )
        .view(-1, 1)
        .requires_grad_(True)
        .to(DEVICE)
    )
    temps = model(ts)
    dT = grad(temps, ts)[0]
    pde = model.r * (Tenv - temps) - dT

    return torch.mean(pde**2)

In [ ]:
netdisc = NetDiscovery(
    1, 1, loss2=physics_loss_discovery, loss2_weight=1, epochs=40000, lr=5e-6
).to(DEVICE)

losses = netdisc.fit(t, T)

Epoch 0/40000, loss: 4816.01
Epoch 4000/40000, loss: 2499.91
Epoch 8000/40000, loss: 2038.04
Epoch 12000/40000, loss: 474.15
Epoch 16000/40000, loss: 3.66
Epoch 20000/40000, loss: 1.70
Epoch 24000/40000, loss: 1.19
Epoch 28000/40000, loss: 0.57
Epoch 32000/40000, loss: 0.37
Epoch 36000/40000, loss: 0.31
